In [10]:
from math import atan2, degrees

In [17]:
class Point:
    def __init__(self, x, y):
        self.x, self.y = float(x), float(y)

    def __eq__(self, other):
        return self.x == other.x and self.y == other.y

    def __ne__(self, other):
        return not self == other

    def __gt__(self, other):
        if self.x > other.x:
            return True
        elif self.x == other.x:
            return self.y > other.y
        return False

    def __lt__(self, other):
        return not self > other

    def __ge__(self, other):
        if self.x > other.x:
            return True
        elif self.x == other.x:
            return self.y >= other.y
        return False

    def __le__(self, other):
        if self.x < other.x:
            return True
        elif self.x == other.x:
            return self.y <= other.y
        return False

    def __add__(self, other):
        return Point(self.x+other.x, self.y+other.y)

    def __sub__(self, other):
        return Point(self.x-other.x, self.y-other.y)

    def __truediv__(self, other):
        return Point(self.x/other, self.y/other)

    def __repr__(self):
        return f"({self.x}, {self.y})"

    def __hash__(self):
        return hash(self.x)

In [12]:
def _construct_points(list_of_tuples):
    points = []
    if list_of_tuples:
        for p in list_of_tuples:
            try:
                points.append(Point(p[0], p[1]))
            except (IndexError, TypeError):
                print(
                    f"Ignoring deformed point {p}. All points"
                    " must have at least 2 coordinates."
                )
    return points


def _validate_input(points):
    if not points:
        raise ValueError(f"Expecting a list of points but got {points}")

    if isinstance(points, set):
        points = list(points)

    try:
        if hasattr(points, "__iter__") and not isinstance(points[0], Point):
            if isinstance(points[0], (list, tuple)):
                points = _construct_points(points)
            else:
                raise ValueError(
                    "Expecting an iterable of type Point, list or tuple. "
                    f"Found objects of type {type(points[0])} instead"
                )
        elif not hasattr(points, "__iter__"):
            raise ValueError(
                f"Expecting an iterable object but got an non-iterable type {points}"
            )
    except TypeError:
        print("Expecting an iterable of type Point, list or tuple.")
        raise

    return points

In [13]:
def _det(a, b, c):
    det = (a.x * b.y + b.x * c.y + c.x * a.y) - (a.y * b.x + b.y * c.x + c.y * a.x)
    return det

def _angle(a, b, c):
    return degrees(atan2( c.y - b.y,  c.x - b.x) - atan2(a.y - b.y, a.x - b.x));

In [14]:
def convex_hull_recursive(points):

    points = sorted(_validate_input(points))
    n = len(points)

    left_most_point = points[0]
    right_most_point = points[n - 1]

    convex_set = {left_most_point, right_most_point}
    upper_hull = []
    lower_hull = []

    for i in range(1, n - 1):
        det = _det(left_most_point, right_most_point, points[i])

        if det > 0:
            upper_hull.append(points[i])
        elif det < 0:
            lower_hull.append(points[i])

    _construct_hull(upper_hull, left_most_point, right_most_point, convex_set)
    _construct_hull(lower_hull, right_most_point, left_most_point, convex_set)

    return sorted(convex_set)


def _construct_hull(points, left, right, convex_set):
    if points:
        extreme_point = None
        extreme_point_distance = float("-inf")
        candidate_points = []

        for p in points:
            det = _det(left, right, p)

            if det > 0:
                candidate_points.append(p)

                if det > extreme_point_distance:
                    extreme_point_distance = det
                    extreme_point = p

        if extreme_point:
            _construct_hull(candidate_points, left, extreme_point, convex_set)
            convex_set.add(extreme_point)
            _construct_hull(candidate_points, extreme_point, right, convex_set)

In [15]:
class Triangle:

    def __init__(self, a, b, c):
        self.a, self.b, self.c = a, b, c

    def __repr__(self):
        return f"{(self.a, self.b, self.c)}"

    def isCoincident(self):
        return self.a == self.b or self.b == self.c or self.c == self.a

# class Triangulate:

def compare(p1, p2):
    if p1 == p2:
        return 0
    elif p1 > p2:
        return 1
    else: return -1

def inCircle(p, t):
    if t.isCoincident():
        return False
    a = t.a - p
    b = t.b - p
    c = t.c - p
    val = (
            (a.x*a.x + a.y*a.y) * (b.x*c.y-c.x*b.y) -
            (b.x*b.x + b.y*b.y) * (a.x*c.y-c.x*a.y) +
            (c.x*c.x + c.y*c.y) * (a.x*b.y-b.x*a.y)
        )
    return val >= 0

def circumCircle(t):
    a,b,c = t
    d = 2 * (a.x * (b.y - c.y) + b.x * (c.y - a.y) + c.x * (a.y - b.y))
    ua = ((a.x * a.x + a.y * a.y) * (b.y - c.y) + (b.x * b.x + b.y * b.y) * (c.y - a.y) + (c.x * c.x + c.y * c.y) * (a.y - b.y)) / d
    ub = ((a.x * a.x + a.y * a.y) * (c.x - b.x) + (b.x * b.x + b.y * b.y) * (a.x - c.x) + (c.x * c.x + c.y * c.y) * (b.x - a.x)) / d
    return (ux, uy)

def triangulate(pxyz):
    # sort vertex array in increasing x values
    pxyz.sort( key = lambda x: x[0])
    """
        Find the maximum and minimum vertex bounds.
        This is to allow calculation of the bounding triangle
    """
    Max = Point(-float('inf'),-float('inf'))
    Min = Point(float('inf'),float('inf'))
    for x,y in pxyz:
        Max.x = max(Max.x, x)
        Max.y = max(Max.y, y)
        Min.x = min(Min.x, x)
        Min.y = min(Min.y, y)

    diff = Max - Min
    dmax = max(diff.x, diff.y)
    mid = (Max+Min)/2

    """
        Set up the supertriangle
        This is a triangle which encompasses all the sample points.
        The supertriangle coordinates are added to the end of the
        vertex list. The supertriangle is the first triangle in
        the triangle list.
    """
    triangles = []

    a = Point(mid.x - 2 * dmax, mid.y - dmax)
    b = Point(mid.x, mid.y + 2 * dmax)
    c = Point(mid.x + 2 * dmax, mid.y - dmax)
    superTriangle = Triangle(a,b,c)
    triangle.append(superTriangle)

    # edge = []
    # for p in pxyz:
    #     edges = []

    """
        Set up the edge buffer.
        If the point (xp,yp) lies inside the circumcircle then the
        three edges of that triangle are added to the edge buffer
        and that triangle is removed.
    """


In [16]:
def main():
    points = [
        (0, 3),
        (2, 2),
        (1, 1),
        (2, 1),
        (3, 0),
        (0, 0),
        (3, 3),
        (2, -1),
        (2, -4),
        (1, -3),
    ]
    # results_recursive = convex_hull_recursive(points)
    # print(results_recursive)

    # a = Point(0,2)
    # b = Point(-2,0)
    # c = Point(2,0)
    # t = Triangle(a,b,c)
    # # print(_angle(a,b,c))
    # p = Point(2, 0)
    # val = inCircle(p, t)
    # print(val)
    print(points)
    triangulate(points)

if __name__ == "__main__":
    main()

[(0, 3), (2, 2), (1, 1), (2, 1), (3, 0), (0, 0), (3, 3), (2, -1), (2, -4), (1, -3)]


NameError: name 'triangle' is not defined